In [18]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [19]:
# Evaluation

with open ('2009091300.txt') as file:
    data=json.load(file)
    print(data.keys())
    print(data['togo'])

dict_keys(['home', 'away', 'drives', 'scrsummary', 'weather', 'media', 'yl', 'qtr', 'note', 'down', 'togo', 'redzone', 'clock', 'posteam', 'stadium'])
0


In [20]:
mydatadict={}
mydatadict['hometeam']=[]
mydatadict['awayteam']=[]
mydatadict['hometeam_score']=[]
mydatadict['awayteam_score']=[]
mydatadict['gamedate']=[]
mydatadict['gamenumber']=[]


for gametype in ['homestats_','awaystats_']:
    for key in ['totfd', 'totyds', 'pyds', 'ryds', 'pen', 'penyds', 'trnovr', 'pt', 'ptyds', 'ptavg', 'top']:
        mydatadict[gametype + key]=[]

mydatadict['homestats_totfd']=[]
mydatadict['awaystats_totfd']=[]

filenames=[filename for filename in os.listdir() if filename.endswith(".txt")]

for filename in filenames:    
    with open (filename) as file:
        data=json.load(file)
        mydatadict['gamedate'].append(datetime.date(int(filename[:4]),int(filename[4:6]),int(filename[6:8])))
        mydatadict['gamenumber'].append(filename[:-4][-2:])
        mydatadict['hometeam'].append(data['home']['abbr'])
        mydatadict['awayteam'].append(data['away']['abbr'])
        mydatadict['hometeam_score'].append(data['home']['score']['T'])
        mydatadict['awayteam_score'].append(data['away']['score']['T'])
        
        for key in ['totfd', 'totyds', 'pyds', 'ryds', 'pen', 'penyds',
                    'trnovr', 'pt', 'ptyds', 'ptavg', 'top']:
            mydatadict['homestats_'+ key].append(data['home']['stats']['team'][key])
            mydatadict['awaystats_'+ key].append(data['away']['stats']['team'][key])


In [21]:
df=pd.DataFrame(mydatadict)

In [22]:
games2018seasondf=df[df.gamedate.between(datetime.date(2018, 9, 6),datetime.date(2018, 12, 30))]
games2018seasondf.to_pickle('games2018seasondf.pickle')

games2017seasondf=df[df.gamedate.between(datetime.date(2017, 9, 7),datetime.date(2017, 12, 31))]
games2017seasondf.to_pickle('games2017seasondf.pickle')

games2016seasondf=df[df.gamedate.between(datetime.date(2016, 9, 8),datetime.date(2017, 1, 1))]
games2016seasondf.to_pickle('games2016seasondf.pickle')

games2015seasondf=df[df.gamedate.between(datetime.date(2015, 9, 10),datetime.date(2016, 1, 3))]
games2015seasondf.to_pickle('games2015seasondf.pickle')

games2014seasondf=df[df.gamedate.between(datetime.date(2014, 9, 4),datetime.date(2014, 12, 28))]
games2014seasondf.to_pickle('games2014seasondf.pickle')

games2013seasondf=df[df.gamedate.between(datetime.date(2013, 9, 5),datetime.date(2013, 12, 29))]
games2013seasondf.to_pickle('games2013seasondf.pickle')

games2012seasondf=df[df.gamedate.between(datetime.date(2012, 9, 5),datetime.date(2012, 12, 30))]
games2012seasondf.to_pickle('games2012seasondf.pickle')

games2011seasondf=df[df.gamedate.between(datetime.date(2011, 9, 8),datetime.date(2012, 1, 1))]
games2011seasondf.to_pickle('games2011seasondf.pickle')